# Content-Based Personalization with LightGBM on Spark

This notebook provides a quick prototype of how to train a [LightGBM](https://github.com/Microsoft/Lightgbm) model on Spark using [MMLSpark](https://github.com/Azure/mmlspark) for a content-based personalization scenario.

We use our dataset which is adapted from the [CRITEO dataset](https://www.kaggle.com/c/criteo-display-ad-challenge), a well known dataset of website ads that can be used to optimize the Click-Through Rate (CTR). Refer to the customer-store interaction PDF for an overview of the dataset contents.

The model is based on [LightGBM](https://github.com/Microsoft/Lightgbm), which is a gradient boosting framework that uses tree-based learning algorithms. Finally, we take advantage of
[MMLSpark](https://github.com/Azure/mmlspark) library, which allows LightGBM to be called in a Spark environment and be computed distributely.

## Global Settings and Imports

In [0]:
!pip install tqdm
!pip install papermill
from azureml.core import Workspace

subscription_id = '796515a0-d9b7-4ab5-9507-440d24feca8e'
resource_group  = 'azure_competition'
workspace_name  = 'workspace_ml'

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print('Library configuration succeeded')
except:
    print('Workspace not found')

Library configuration succeeded

In [0]:
import os
import sys

sys.path.append("../../")

import pyspark
from pyspark.ml import PipelineModel
from pyspark.ml.feature import FeatureHasher
import papermill as pm

from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.common.notebook_utils import is_databricks
from reco_utils.dataset.criteo import load_spark_df
from reco_utils.dataset.spark_splitters import spark_random_split

# Setup MML Spark
if not is_databricks():
    # get the maven coordinates for MML Spark from databricks_install script
    from scripts.databricks_install import MMLSPARK_INFO
    packages = [MMLSPARK_INFO["maven"]["coordinates"]]
    repo = MMLSPARK_INFO["maven"].get("repo")
    spark = start_or_get_spark(packages=packages, repository=repo)
    dbutils = None
    print("MMLSpark version: {}".format(MMLSPARK_INFO['maven']['coordinates']))

from mmlspark.train import ComputeModelStatistics
from mmlspark.lightgbm import LightGBMClassifier

print("System version: {}".format(sys.version))
print("PySpark version: {}".format(pyspark.version.__version__))

System version: 3.7.3 (default, Feb 20 2020, 02:03:03) 
[GCC 5.4.0 20160609]
PySpark version: 2.4.6.dev0

In [0]:
# Prototype data size, it can be "sample" or "full"
DATA_SIZE = "sample"

# LightGBM parameters
# More details on parameters: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
NUM_LEAVES = 32
NUM_ITERATIONS = 50
LEARNING_RATE = 0.1
FEATURE_FRACTION = 0.8
EARLY_STOPPING_ROUND = 10

# Model name
MODEL_NAME = 'lightgbm_prototype.mml'

## Data Preparation

The dataset contains 4 labels. The label is multi-class depending on the category of product the customer has most affinity towards. Please refer to customer-store interaction PDF for an overview of the dataset. 

In [0]:
# File location and type
file_location = "/FileStore/tables/synth_data.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
raw_data = spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
raw_data.limit(2).toPandas().head()

,_c0,customer_ID,category,y,pur_value,city
0,0,17850.0,Video Game Accessories,1,255.0,Montreal
1,1,17850.0,Other,1,339.0,Edmonton


In [0]:
raw_data.createOrReplaceTempView("synth_data")

### Feature Processing
First, the dataset is splitted randomly for training and testing and feature processing is applied to each dataset.

In [0]:
raw_train, raw_test = spark_random_split(raw_data, ratio=0.8, seed=42)

In [0]:
columns = [c for c in raw_data.columns if c != 'y']
feature_processor = FeatureHasher(inputCols=columns, outputCol='features')

In [0]:
train = feature_processor.transform(raw_train)
test = feature_processor.transform(raw_test)

## Model Training
In MMLSpark, the LightGBM implementation for binary classification is invoked using the `LightGBMClassifier` class and specifying the objective as `"multiclass"`. <br><br>

### Hyper-parameters
Below are some of the key [hyper-parameters](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters-Tuning.rst) for training a LightGBM classifier on Spark:
- `numLeaves`: the number of leaves in each tree
- `numIterations`: the number of iterations to apply boosting
- `learningRate`: the learning rate for training across trees
- `featureFraction`: the fraction of features used for training a tree
- `earlyStoppingRound`: round at which early stopping can be applied to avoid overfitting

In [0]:
lgbm = LightGBMClassifier(
    labelCol="y",
    featuresCol="features",
    objective="multiclass",
    isUnbalance=False,
    boostingType="gbdt",
    boostFromAverage=True,
    baggingSeed=42,
    numLeaves=NUM_LEAVES,
    numIterations=NUM_ITERATIONS,
    learningRate=LEARNING_RATE,
    featureFraction=FEATURE_FRACTION,
    earlyStoppingRound=EARLY_STOPPING_ROUND
)

### Model Training and Evaluation

In [0]:
model = lgbm.fit(train)
predictions = model.transform(test)

In [0]:
evaluator = (
    ComputeModelStatistics()
    .setScoredLabelsCol("prediction")
    .setLabelCol("y")
    .setEvaluationMetric("classification")
)

result = evaluator.transform(predictions)
# auc = result.select("confusion_matrix").collect()[0][0]
result.show()

+---------------+--------------------+------------------+------------------+------------------+------------------+------------------------+---------------------+
evaluation_type| confusion_matrix| accuracy| precision| recall| average_accuracy|macro_averaged_precision|macro_averaged_recall|
+---------------+--------------------+------------------+------------------+------------------+------------------+------------------------+---------------------+
 Classification|8153.0 1.0 31...|0.8309816121326593|0.8309816121326593|0.8309816121326593|0.9323926448530637| 0.8270688979196852| 0.4217247010379522|
+---------------+--------------------+------------------+------------------+------------------+------------------+------------------------+---------------------+

## Model Saving 
The full pipeline for operating on raw data including feature processing and model prediction can be saved and reloaded for use in another workflow.

In [0]:
# save model
pipeline = PipelineModel(stages=[feature_processor, model])
pipeline.write().overwrite().save(MODEL_NAME)

In [0]:
# cleanup spark instance
if not is_databricks():
    spark.stop()

## Additional Reading
\[1\] Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. LightGBM: A highly efficient gradient boosting decision tree. In Advances in Neural Information Processing Systems. 3146–3154. https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf <br>
\[2\] MML Spark: https://mmlspark.blob.core.windows.net/website/index.html <br>